In [1]:
# This program require the following from Python:
import pandas as pd

In [2]:
# This program also refers to an number of programs in a separate file called DataFunctions.py. 
# This files contains the following functions
#     GetDatWB(WBcode, years) - extracts data from World Bank using code and year/s. This is used the get data like GDP, remittances etc
#     GetDatIMFex(filename, indicator, BOPType, years) - extracts receipts and payments from IMF Balance of payments data downloaded into an excel (e.g., workers compensation).
#         IMF does have an API but could not get it to work
#     DatFill(df, mappingfile, fill) - fills a dataframe (df) using other data fill (e.g., fill remittances using GDP).
#     DatFillEq(Paid, Rec, mappingfile, fill) - fills two data frames and ensures they are equal.  For instance remittances 
#         paid andreceived by country are filled using a data series named fill and are also scaled to ensure payments equal receipts. 
#         Note this function has 3 outputs - updated payments, receipts and the scale.  The closer scale is to 1 the less scaling done to make sure payments equal receipts.
import DataFunctions as DF

In [3]:
# Year for which we need data
year = 2019
# Ideally this could be an input
# year = int(input("Please enter the year: "))

In [4]:
# Gets POP from World Bank API
POP = DF.GetDatWB('SP.POP.TOTL',[year])

<class 'pandas.DataFrame'>
Index: 265 entries, ABW to ZWE
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    265 non-null    float64
dtypes: float64(1)
memory usage: 4.1+ KB


In [5]:
# Gets GDP from World Bank API
GDP = DF.GetDatWB('NY.GDP.MKTP.CD',[year])

<class 'pandas.DataFrame'>
Index: 262 entries, ABW to ZWE
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    262 non-null    float64
dtypes: float64(1)
memory usage: 4.1+ KB


In [6]:
# Fill Taiwan's data for 2019 (need to update if update year - can usually find in google) 
GDP.loc['TWN'] = 613512000000
POP.loc['TWN'] = 23600000

In [7]:
# Fill GDP so as to avoid missing tiny countries
GDP = DF.DatFill(GDP, 'GTAPMap.xlsx', POP)

In [8]:
# Gets Worker's compensation payments for IMF excel file
WCompPaid = DF.GetDatIMFex(filename = 'dataset_2026-02-11WComp.xlsx', indicator = 'Compensation of employees', BOPType = 'DB_T', years = [year])

<class 'pandas.DataFrame'>
Index: 181 entries, USA to DEU
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    178 non-null    float64
dtypes: float64(1)
memory usage: 2.8+ KB


In [9]:
# Gets Worker's compensation payments for IMF excel file
WCompRec = DF.GetDatIMFex(filename = 'dataset_2026-02-11WComp.xlsx', indicator = 'Compensation of employees', BOPType = 'CD_T', years = [year])

<class 'pandas.DataFrame'>
Index: 183 entries, DNK to NGA
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    181 non-null    float64
dtypes: float64(1)
memory usage: 2.9+ KB


In [10]:
# Fills workers compensation data and ensure receipts equal payments
WCompPaid, WCompRec, scale = DF.DatFillEq(WCompPaid, WCompRec, 'Mappings.xlsx', GDP)
# Print how much scaling was required
print("This is how much scaling was required (1 means no scaling):", scale)

This is how much scaling was required (1 means no scaling): 0.9740727595529727


In [11]:
# Aggregate to GTAP using mapping file
WCompPaid = DF.DatAgg(WCompPaid,'GTAPMap.xlsx')
WCompRec = DF.DatAgg(WCompRec,'GTAPMap.xlsx')

In [12]:
# Gets Primary income from World Bank API
PIPaid = DF.GetDatWB('BM.GSR.FCTY.CD',[year]) 

<class 'pandas.DataFrame'>
Index: 202 entries, ABW to ZWE
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    202 non-null    float64
dtypes: float64(1)
memory usage: 3.2+ KB


In [13]:
# Gets Primary income from World Bank API
PIRec = DF.GetDatWB('BX.GSR.FCTY.CD',[year]) 

<class 'pandas.DataFrame'>
Index: 202 entries, ABW to ZWE
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    202 non-null    float64
dtypes: float64(1)
memory usage: 3.2+ KB


In [14]:
# Fills primary income data and ensure receipts equal payments
PIPaid, PIRec, scale = DF.DatFillEq(PIPaid, PIRec, 'Mappings.xlsx',GDP)
# Print how much scaling was required
print("This is how much scaling was required (1 means no scaling):", scale)

This is how much scaling was required (1 means no scaling): 1.0044330681794005


In [15]:
# Aggregate to GTAP using mapping file
PIPaid = DF.DatAgg(PIPaid,'GTAPMap.xlsx')
PIRec = DF.DatAgg(PIRec,'GTAPMap.xlsx')

In [16]:
# Gets Remittances from World Bank API
RemRec = DF.GetDatWB('BX.TRF.PWKR.CD.DT',[year])

<class 'pandas.DataFrame'>
Index: 245 entries, ABW to ZWE
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    245 non-null    float64
dtypes: float64(1)
memory usage: 3.8+ KB


In [17]:
# Gets Remittances from World Bank API
RemPaid = DF.GetDatWB('BM.TRF.PWKR.CD.DT',[year])

<class 'pandas.DataFrame'>
Index: 247 entries, ABW to ZWE
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    247 non-null    float64
dtypes: float64(1)
memory usage: 3.9+ KB


In [18]:
# Fills remittances data and ensure receipts equal payments
RemPaid, RemRec, scale = DF.DatFillEq(RemPaid, RemRec, 'Mappings.xlsx',GDP)
# Print how much scaling was required
print("This is how much scaling was required (1 means no scaling):", scale)

This is how much scaling was required (1 means no scaling): 0.6884659105954026


In [19]:
# Aggregate to GTAP using mapping file
RemPaid = DF.DatAgg(RemPaid,'GTAPMap.xlsx')
RemRec = DF.DatAgg(RemRec,'GTAPMap.xlsx')

In [20]:
# Gets Aid from World Bank API
AidRec = DF.GetDatWB('DT.ODA.ODAT.CD',[year])

<class 'pandas.DataFrame'>
Index: 218 entries, ABW to ZWE
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    218 non-null    float64
dtypes: float64(1)
memory usage: 3.4+ KB


In [21]:
# Gets Aid from World Bank API
AidPaid = DF.GetDatWB('DC.ODA.TOTL.CD',[year])

<class 'pandas.DataFrame'>
Index: 32 entries, AUS to USA
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2019    32 non-null     float64
dtypes: float64(1)
memory usage: 512.0+ bytes


In [22]:
# Fills aid data and ensure receipts equal payments
AidPaid, AidRec, scale = DF.DatFillEq(AidPaid, AidRec, 'Mappings.xlsx',GDP)
# Print how much scaling was required
print("This is how much scaling was required (1 means no scaling):", scale)

This is how much scaling was required (1 means no scaling): 1.495464588457991


In [23]:
# Aggregate to GTAP using mapping file
AidPaid = DF.DatAgg(AidPaid,'GTAPMap.xlsx')
AidRec = DF.DatAgg(AidRec,'GTAPMap.xlsx')

In [24]:
# Note we want foreign income related to workers and capital - Primary income includes workers compensation so it is subtracted.
InvPaid = PIPaid - WCompPaid
InvRec = PIRec - WCompRec

In [25]:
# Write data to excel
with pd.ExcelWriter("MyGTAPoutput.xlsx", engine="openpyxl") as writer:
    RemPaid.to_excel(writer, sheet_name="RemPaid", index=True)
    RemRec.to_excel(writer, sheet_name="RemRec", index=True)
    InvPaid.to_excel(writer, sheet_name="InvPaid", index=True)
    InvRec.to_excel(writer, sheet_name="InvRec", index=True)
    AidPaid.to_excel(writer, sheet_name="AidPaid", index=True)
    AidRec.to_excel(writer, sheet_name="AidRec", index=True)